Instead of testing all combinations, Random Search randomly selects hyperparameters, making it much faster.

✅ Pros: Faster than Grid Search.
❌ Cons: May not always find the best parameters.

In [47]:
import numpy as np

from xgboost import XGBClassifier

from sklearn.datasets import make_classification

from sklearn.model_selection import RandomizedSearchCV, train_test_split

from sklearn.metrics import accuracy_score,classification_report

In [48]:
# Generate synthetic data

X,y = make_classification(n_samples=500, n_features=10, n_informative=2, n_redundant=2, random_state=42)

In [49]:
# Split the data

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=42)

In [50]:
# Initialize model

xgb_model = XGBClassifier(eval_metric = 'logloss')

In [51]:
# Define parameter grid with distributions

param_distributions = {

    'n_estimators' : np.arange(50 , 300, 50),
    'learning_rate' : np.linspace(0.01 , 0.3, 5),
    'max_depth' : np.arange(3 , 10, 2),
    'subsample' : np.linspace(0.5 , 1.0, 5),
    'colsample_bytree' : np.linspace(0.5 , 1.0, 5)
}

In [52]:
# Random Search

random_search = RandomizedSearchCV(
    xgb_model,  
    param_distributions = param_distributions,
    n_iter=10,
    scoring="accuracy",
    cv=3,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning...
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': array([0.5  , 0.625, 0.75 , 0.875, 1.   ]),
                                        'learning_rate': array([0.01  , 0.0825, 0.155 , 0.2275, 0.3   ]),
                                        'max_depth': array([3, 5, 7, 9]),
                                        'n_estimators': array([ 50, 100, 150, 200, 250]),
                                        'subsample': array([0.5  , 0.625, 0.75 , 0.875, 1.   ])},
                   scoring='accuracy', verbose=1)

In [53]:
# Best Parameters

print(f"Best Parameters : {random_search.best_params_}")

Best Parameters : {'subsample': 0.875, 'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.08249999999999999, 'colsample_bytree': 1.0}


In [54]:
# Evaluate the best model

best_model = random_search.best_estimator_

y_pred = best_model.predict(X_test)


print(f"Accuracy Score : {accuracy_score(y_test,y_pred)}")

print("Classification Report \n:")

print(classification_report(y_test, y_pred))

Accuracy Score : 0.9266666666666666
Classification Report 
:
              precision    recall  f1-score   support

           0       0.96      0.90      0.93        79
           1       0.89      0.96      0.93        71

    accuracy                           0.93       150
   macro avg       0.93      0.93      0.93       150
weighted avg       0.93      0.93      0.93       150

